In [43]:
from groq import Groq
import yaml
import PyPDF2
import ast

In [4]:
with open('env_variables.yml', 'r') as file:
    env_variables = yaml.safe_load(file)

In [8]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

In [5]:
client = Groq(api_key=env_variables['GROQ_API_KEY'])

In [10]:
# Extraer el texto del PDF
pdf_text = extract_text_from_pdf("./data/programas/Historia_sexto_primaria.pdf")

In [11]:
len(pdf_text)

51537

In [21]:


# Crear el mensaje para el modelo
completion = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": """Eres un sistema que diseña planes pedagógicos experto. El nivel del público objetivo es niños de sexto de primaria en méxico. Tu tarea es diseñar planes de studio conforme a un esquema
            de enseñanza homologado por el gobierno de méxico. Diseñarás planes de estudio, tal que la información que te proporcionemos se pueda enseñar en un lapso de 18 semanas, por lo que es importante que
            identifiques que temas necesitan mas semanas y que temas son mas simples. Para cada uno devolverás el numero de semana en el cual será impartido y el tema, seguido de puntos principales a tocar en cada día de la semana.
            Te compartiremos como contexto un texto que representa todo el programa del ciclo escolar. Desde Propósitos, Enfoque didáctico, organización de los aprendizajes, y tablas de aprendizajes esperados y el contenido que debes utilizar.
            Te muestro un ejemplo de salida:
            
            {"Semana 1": {
                        "Contenido": "Tema de la semana",
                        "Aprendizajes clave": Lista de aprendizajes necesarios. Al menos 5 para cada día de la semana}}

            Es muy importante que respetes este formato ya que tu salida debe ser evaluada como un diccionario de python.

            
            """
        },
        {
            "role": "user",
            "content": f"Este es el contenido del PDF del programa para sexto grado de primaria para historia {pdf_text}. Ahora, hazme el plan de estudios en el formato solicitado para 20 semanas sobre la materia **historia**"
        }
    ],
    temperature=1.2,
    max_tokens=8000,
    top_p=1,
    stream=False,
    stop=None,
)


In [24]:
output=completion.choices[0].message.content

In [41]:
clean=output[61:-7]

In [55]:
text_file = open("lessons.txt", "w")

text_file.write(clean)

text_file.close()